[Reference](https://medium.com/biased-algorithms/a-step-by-step-guide-to-federated-learning-in-computer-vision-0984e4a7f8d5)

# Step 1: Setting Up the Environment

In [1]:
!pip install tensorflow tensorflow-federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 367.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 37.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to redu

In [2]:
!pip install torch syft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00
ERROR: Operation cancelled by user


# Step 2: Preparing the Dataset

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Function to partition the dataset into non-IID subsets
def create_noniid_partitions(x_data, y_data, num_clients=10):
    # Create non-IID partitions by randomly selecting class-specific samples for each client
    partitions = []
    num_classes = len(np.unique(y_data))

    for i in range(num_clients):
        indices = np.array([], dtype=int)
        for c in range(num_classes):
            # Select a subset of samples from each class (non-uniformly)
            class_indices = np.where(y_data == c)[0]
            chosen_indices = np.random.choice(class_indices, size=int(len(class_indices) / num_clients), replace=False)
            indices = np.concatenate([indices, chosen_indices])

        partitions.append((x_data[indices], y_data[indices]))

    return partitions

# Simulate 10 clients with non-IID data
client_partitions = create_noniid_partitions(x_train, y_train, num_clients=10)

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Step 3: Defining the Model

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def create_cnn_model():
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  # 10 classes for CIFAR-10
    ])
    return model

ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.10/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


# Step 4: Training the Model Locally

In [2]:
def train_local_model(model, train_data, train_labels, epochs=5, batch_size=32):
    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model locally
    model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, verbose=1)

    return model

# Step 5: Aggregating Model Updates

In [3]:
def aggregate_weights(client_weights):
    # Initialize an empty list to store the averaged weights
    avg_weights = []

    # Zip all weights together and compute the mean for each layer
    for weights in zip(*client_weights):
        avg_weights.append(np.mean(weights, axis=0))

    return avg_weights

# Step 6: Model Validation and Testing

In [4]:
def validate_model(model, x_val, y_val):
    loss, accuracy = model.evaluate(x_val, y_val, verbose=0)
    print(f"Validation Loss: {loss}")
    print(f"Validation Accuracy: {accuracy}")

# Step 7: Deployment of the Federated Model

In [6]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [7]:
# Convert to a quantized TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()

# Save the quantized model
with open('quantized_model.tflite', 'wb') as f:
    f.write(quantized_model)

In [8]:
def weighted_average(client_weights, client_sizes):
    total_size = sum(client_sizes)
    weighted_updates = [
        w * (size / total_size) for w, size in zip(client_weights, client_sizes)
    ]
    return sum(weighted_updates) / len(client_weights)

In [9]:
# Train for multiple epochs locally before sending updates
def federated_training(client_model, local_data, num_local_epochs=5):
    for epoch in range(num_local_epochs):
        client_model.fit(local_data, epochs=1)
    return client_model.get_weights()

In [10]:
class TrainingLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch}: loss = {logs['loss']}, accuracy = {logs['accuracy']}")

# Apply logger during local training
client_model.fit(local_data, epochs=5, callbacks=[TrainingLogger()])